In [ ]:
# 📦 herunterladen nötige Bibliotheken
%pip install pymupdf langchain chromadb sentence-transformers --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import fitz  # PyMuPDF

pdf_path = "hai_ai_index_report_2025.pdf"  # PDF datei

doc = fitz.open(pdf_path)
raw_text = "".join([page.get_text() for page in doc])
print("Toplam karakter sayısı:", len(raw_text))


Toplam karakter sayısı: 822820


In [15]:
import fitz
doc = fitz.open(pdf_path)
raw_text = ''.join([page.get_text() for page in doc])
print(f"Toplam karakter sayısı: {len(raw_text)}")


Toplam karakter sayısı: 822820


In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = text_splitter.split_text(raw_text)

print(f"Chunk sayısı: {len(chunks)}")


Chunk sayısı: 1033


In [17]:
%pip install -U langchain-community
from langchain.docstore.document import Document

documents = []
for i, chunk in enumerate(chunks):
    documents.append(Document(page_content=chunk, metadata={"chunk_id": i}))


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# 🔤 Embedding modeli: macht Text zu vektiörische Zahlen
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# 💾 ChromaDB ile vektör veritabanı oluştur
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_function,
    persist_directory="rag_index"  # Speichern in Datei
)

# 📌 bring fest Basedatei
db.persist()


In [ ]:
%pip install openai
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "sk-..."  

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableConfig

memory = ConversationBufferMemory()
config = RunnableConfig(memory=memory)


In [21]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma(
    persist_directory="rag_index",
    embedding_function=embedding_function
)


In [22]:
from langchain.chains import ConversationalRetrievalChain

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory
)


In [23]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory,
    return_source_documents=False  # ✅ sadece 'answer' döner, hata vermez
)


In [ ]:
# ohne Retrieval: Antwort mit nur Modell
q = "What are the main AI investment trends in 2024?"
print("[Retrieval geschlossen]")
print(llm.predict(q))


[Retrieval KAPALI]


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-.... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

: 

In [ ]:
# mit Retrieval: anhand Quelle
print("\n[Retrieval AÇIK]")
print(conversation_chain.run(q))


In [ ]:
# Dieselbe Frage kann mit unterschiedlichen System-Prompts getestet werden.
# (Direkt mit ChatOpenAI wird nicht unterstützt, falls eine Custom-Chain benötigt wird, kann dies später umgesetzt werden)
# Als Beispiel verwenden wir drei Versionen, die unterschiedlich formuliert sind:
variations = [
    "What happened in AI investments last year?",
    "Give me key trends in artificial intelligence funding in 2024.",
    "Summarize AI investment highlights for 2024."
]

for i, v in enumerate(variations):
    print(f"\nPrompt #{i+1}: {v}")
    print(conversation_chain.run(v))
